In [4]:
import pandas as pd
import json
from typing import List, Dict

In [5]:
base_df = pd.read_csv("/projects/asankaranarayana/Fine_Tune_Hackathon/v1/new_data.csv")

In [6]:
send_money_wtout_note = base_df[base_df['intent']=='send_money'][['Prompt','entities']]

In [7]:
len(send_money_wtout_note)

5000

In [8]:
from appworld import AppWorld, load_task_ids
import os

os.environ['APPWORLD_ROOT'] = '/projects/asankaranarayana/Fine_Tune_Hackathon/appworld'
task_ids = load_task_ids("train") # loads train ids, other options: dev, test_normal, test_challenge
task_id = task_ids[0]
world = AppWorld(task_id=task_id)

In [5]:
verbs = ["Send", "Transfer", "Pay", "Shoot over", "Wire", "Send over", "Venmo"]
amount_formats = [
    lambda amt: f"${amt}",
    lambda amt: f"{amt} bucks",
    lambda amt: f"{amt}",
    lambda amt: "a fiver",
    lambda amt: f"{amt} quid", 
]
recipients = ['Ronald','Jerry','Michael','Brenda','Emily','Gary','Jason','Daniel','Rebecca','Jessica','Carlos','Edward','Patricia','Steven','James','Patrick','Ravi','Maria','Melissa','Jennifer','Katherine','Debra','Sarah','Paul','Amy','Christine','Fatima','Deborah','Matthew','Gregory','Donald','Sharon','Ashley','Amanda','Carolyn','George','Lisa','Joshua','Jeffrey','Priya','Sandra','Charles','William','Samantha','Eric','Cynthia','John','Richard','Carol','Ruth','Zoe','Jack','Ali','Angela','Nicole','Stephanie','Dennis','Emma','Karen','Robert','Elizabeth','Mike','Joseph','Larry','Justin','Barbara','Anthony','Liam','Kimberly','Rachel','Catherine','Michelle','Stephen','Christopher','Raymond','Alexander','Kevin','Susan','Jacob','Nicholas','Linda','Frank','Janet','Scott','Brian','Jonathan','Kenneth','David','Benjamin','Betty','Mary','Brandon','Ryan','Thomas','Heather','Nancy','Timothy','Mark','Donna','Laura','Pamela','Kathleen','Shirley','Samuel','Anna','Helen','Andrew','Margaret']
purposes = [
    "groceries", "lunch", "rent", "utilities", "gas", "books", "concert tickets",
    "Uber", "coffee", "gym", "childcare", "movie night", "gift", "birthday party", "snacks",
    "electric bill", "phone bill", "internet", "car payment", "parking", "dinner", "flight tickets",
    "hotel stay", "pet supplies", "medicines", "cleaning service", "home repairs",
    "haircut", "babysitting", "dry cleaning", "event registration", "furniture",
    "tutoring", "school fees", "donation", "weekend trip",
    "bike repairs", "dentist visit", "birthday gift", "TV subscription",
    "shared groceries", "pool cleaning", "birthday dinner", "pizza night", "game night snacks",
    "gas money", "laundry", "beach trip", "ski trip", "Christmas gift",
    "New Year party", "group dinner", "drinks", "concert parking", "moving help", "car wash",
    "shared taxi", "brunch", "bachelorette party", "farewell gift", "work lunch",
    "Amazon purchase", "online course", "dog walking", 
    "grocery delivery", 
    "anniversary cake", "Airbnb", 
    "costume rental", "theme park tickets", "museum tickets", "ice cream run", "spa appointment",
    "massages", "team lunch", "college reunion", "kids’ school trip", "new tires", "engine oil change"
]
templates = [
    "{verb} {amount_str} to {name} for {purpose}",
    "{verb} {amount_str} to {name} – {purpose}",
    "{verb} {amount_str} for {purpose} to {name}",
    "{verb} {amount_str} to cover {purpose} with {name}",
    "{verb} {amount_str} their way for {purpose}",
    "Can you {verb_lower} {amount_str} to {name} for {purpose}?",
    "{name} needs {amount_str} for {purpose}, please {verb_lower} it",
]


def generate_example():
    verb = random.choice(verbs)
    verb_lower = verb.lower()
    name = random.choice(recipients)
    purpose = random.choice(purposes)
    
    
    amt_value = random.choice([5, 10, 15, 20, 25, 30, 40, 50, 75, 100, 150, 200, 91, 85, 64, 38, 105, 128, 47, 55, 79, 115, 124, 178])
    amt_format = random.choice(amount_formats)
    amount_str = amt_format(amt_value)

    
    if amount_str == "a fiver":
        amount_entity = 5
    elif "quid" in amount_str:
        amount_entity = int(''.join(filter(str.isdigit, amount_str)))
    elif "bucks" in amount_str:
        amount_entity = int(''.join(filter(str.isdigit, amount_str)))
    else:
        try:
            amount_entity = int(amount_str.replace("$", "").replace("USD", "").strip())
        except:
            amount_entity = None

    template = random.choice(templates)
    sentence = template.format(verb=verb, verb_lower=verb_lower, amount_str=amount_str, name=name, purpose=purpose)

    entities = {
        "amount": amount_entity,
        "recipient": name,
        "note": purpose
    }
    return sentence, entities


examples = [generate_example() for _ in range(5000)]

df = pd.DataFrame(examples, columns=["Prompt", "entities"])

NameError: name 'random' is not defined

In [57]:
df.to_csv("/projects/asankaranarayana/Fine_Tune_Hackathon/v1/synthetic_data.csv", index=False)

### Sending money

In [9]:
api_list = world.apis.api_docs.show_api_descriptions(app_name='venmo')

In [5]:
api_list

[{'name': 'show_account',
  'description': 'Show your account information. Unlike show_profile, this includes private information.'},
 {'name': 'signup', 'description': 'Sign up to create account.'},
 {'name': 'delete_account', 'description': 'Delete your account.'},
 {'name': 'update_account_name',
  'description': 'Update your first or last name in the account profile.'},
 {'name': 'login', 'description': 'Login to your account.'},
 {'name': 'logout', 'description': 'Logout from your account.'},
 {'name': 'send_verification_code',
  'description': 'Send account verification code to your email address.'},
 {'name': 'verify_account',
  'description': 'Verify your account using the verification code sent to your email address.'},
 {'name': 'send_password_reset_code',
  'description': 'Send password reset code to your email address.'},
 {'name': 'reset_password',
  'description': 'Reset your password using the password reset code sent to your email address.'},
 {'name': 'show_profile',
 

In [10]:
create_transaction_details = world.apis.api_docs.show_api_doc(app_name='venmo', api_name='create_transaction')

In [11]:
search_friends_details = world.apis.api_docs.show_api_doc(app_name='venmo', api_name='search_friends')

In [12]:
from itertools import combinations
import random
import string

def create_email(name):
    domain = "@example.com"
    random_sub_str = ''.join(random.choice(string.ascii_lowercase) for _ in range(3))
    full_str = name+random_sub_str

    email_length = random.randint(len(name),len(full_str))
    for ea in combinations(full_str,email_length):
        email_address = ''.join(ea)+domain
    
    return email_address

In [13]:
user_queries = []
for i in range(len(send_money_wtout_note)):
    query = {
        "user_query": send_money_wtout_note.iloc[i]['Prompt'].strip(),
        "slots":json.loads(send_money_wtout_note.iloc[i]['entities'].strip())
    }
    user_queries.append(query)

In [14]:
synthetic_data = pd.read_csv("/projects/asankaranarayana/Fine_Tune_Hackathon/v1/synthetic_data.csv")

In [15]:
for i in range(len(synthetic_data)):
    query = {
        "user_query": send_money_wtout_note.iloc[i]['Prompt'].strip(),
        "slots":json.loads(send_money_wtout_note.iloc[i]['entities'].strip())
    }
    user_queries.append(query)

In [ ]:
# {"role": "system", "content": """You are a super intelligent AI Assistant whose job is to achieve my day-to-day tasks completely autonomously.
#         To do this, you will need to interact with app/s (e.g., spotify, venmo, etc) using their associated APIs on my behalf. For this you will undertake a *multi-step conversation* using a python REPL environment. That is, you will write the python code and the environment will execute it and show you the result, based on which, you will write python code for the next step and so on, until you've achieved the goal. This environment will let you interact with app/s using their associated APIs on my behalf.

# Here are three key APIs that you need to know to get more information

# # To get a list of apps that are available to you.
# print(apis.api_docs.show_app_descriptions())

# # To get the list of apis under any app listed above, e.g. supervisor
# print(apis.api_docs.show_api_descriptions(app_name='supervisor'))

# # To get the specification of a particular api, e.g. supervisor app's show_account_passwords
# print(apis.api_docs.show_api_doc(app_name='supervisor', api_name='show_account_passwords'))

# Each code execution will produce an output that you can use in subsequent calls. Using these APIs, you can now generate code, that the environment will execute, to solve the task.

# **Key instructions and disclaimers**:
# 1. Only generate valid code blocks, i.e., do not put them in ```...``` or add any extra formatting. Any thoughts should be put as code comments.
# 2. You can use the variables from the previous code blocks in the subsequent code blocks.
# 3. Write small chunks of code and only one chunk of code in every step. Make sure everything is working correctly before making any irreversible change.
# 4. The provided Python environment has access to its standard library. But modules and functions that have a risk of affecting the underlying OS, file system or process are disabled. You will get an error if do call them.
# 5. To interact with apps, only use the provided APIs, and not the corresponding Python packages. E.g., do NOT use `spotipy` for Spotify. Remember, the environment only has the standard library.
# 6. The provided API documentation has both the input arguments and the output JSON schemas. All calls to APIs and parsing its outputs must be as per this documentation.
# 7. For APIs that return results in "pages", make sure to consider all pages.
# 8. All my personal information, and information about my app account credentials, physical addresses and owned payment cards are stored in the "supervisor" app. You can access them via the APIs provided by the supervisor app.
# 9. Once you have completed the task, call `apis.supervisor.complete_task()`. If the task asks for some information, return it as the answer argument, i.e. call `apis.supervisor.complete_task(answer=<answer>)`. For tasks that do not require an answer, just skip the answer argument or pass it as None.
# 10. The answers, when given, should be just entity or number, not full sentences, e.g., `answer=10` for "How many songs are in the Spotify queue?". When an answer is a number, it should be in numbers, not in words, e.g., "10" and not "ten".
# 11. You can also pass `status="fail"` in the complete_task API if you are sure you cannot solve it and want to exit.
# 12. You must make all decisions completely autonomously and not ask for any clarifications or confirmations from me or anyone else.
# """},


# messages = [
        
        # {"role": "user", "content": user_msg},
#         {"role": "assistant", "content": """#First, let's check the available APIs for the Venmo app to see how we can send money.
# answer = world.apis.api_docs.show_api_descriptions(app_name='venmo')"""},
        # {"role": "user", "content": str(api_list)},
#         {"role": "assistant", "content": """#The 'create_transaction' API seems to be the one we need to send money. Let's check its specifications.
# answer = world.apis.api_docs.show_api_doc(app_name='venmo', api_name='create_transaction')"""},
        # {"role": "user", "content": str(create_transaction_details)},
#         {"role":"assistant","content":"""#Let's proceed to send ${amt} to {rcvr}. We need to find the recipient's email. Let's check the list of APIs to see how we can get contact details of a user.
# answer = world.apis.api_docs.show_api_descriptions(app_name='venmo')""".format(amt = amount,rcvr = recipient)},
        # {"role": "user", "content": str(api_list)},
#         {"role": "assistant", "content": """#The 'search_friends' API seems to be the one we need to get the recipient's email. Let's check its specifications.
# answer = world.apis.api_docs.show_api_doc(app_name='venmo', api_name='search_friends')"""},
        # {"role": "user", "content": str(search_friends_details)},
#         {"role": "assistant", "content": """#Let's find {rcvr}'s email using the 'serach_friends' API.
# answer = world.apis.spotify.search_friends(access_token = access_token, query = "{rcvr}")""".format(rcvr = recipient)},
#         {"role": "user", "content": str({'first_name': recipient,
# 'last_name': '',
# 'email': email,
# 'registered_at': '',
# 'friends_since': ''})}
    # ]
    
    # try:
#         note = example['slots']['note']
#         temp = [
#         {"role": "assistant", "content": """#{rcvr}'s email is {em}. Let's proceed to send ${amt} for {nt} using the 'create_transaction' API.
# answer = world.apis.venmo.create_transaction( receiver_email='{em}', amount={amt}, description={nt}))""".format(rcvr = recipient, em = email, amt = amount, nt = note)},
        # {"role": "user", "content": str({"message":"Sent money", "transaction_id": ""})},
#         {"role": "assistant", "content": """#The transaction was successful. Now, let's mark the task as complete.
# answer = world.apis.supervisor.complete_task()"""}
#         ]
    
    # except:
    #     temp = [
#         {"role": "assistant", "content": """#{rcvr}'s email is {em}. Let's proceed to send ${amt} using the 'create_transaction' API.
# answer = world.apis.venmo.create_transaction( receiver_email='{em}', amount={amt}))""".format(rcvr = recipient, em = email, amt = amount)},
        # {"role": "user", "content": str({"message":"Sent money", "transaction_id": ""})},
#         {"role": "assistant", "content": """#The transaction was successful. Now, let's mark the task as complete.
# answer = world.apis.supervisor.complete_task()"""}
#         ]
    
    
    # final_messages = messages+temp
    

In [19]:
# trajectory for sending money without a note
def generate_trajectory(example):
    amount = example['slots']['amount']
    recipient = example['slots']['recipient']
    user_msg = example['user_query']
    email = create_email(recipient)
    
    message_list = [
        {"messages": [{"role": "user", "content": """User Instruction: {q}""".format(q = user_msg)}, 
                      {"role": "assistant", "content": """#First, let's check the available APIs for the Venmo app to see how we can send money.
answer = world.apis.api_docs.show_api_descriptions(app_name='venmo')"""}]},
        
        {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
{apis}""".format(q = user_msg, apis = api_list)}, 
                      {"role": "assistant", "content": """#The 'create_transaction' API seems to be the one we need to send money. Let's check its specifications.
answer = world.apis.api_docs.show_api_doc(app_name='venmo', api_name='create_transaction')"""}]},
        
        {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
{txn}""".format(q = user_msg, txn = create_transaction_details)}, 
                      {"role": "assistant", "content": """#Let's proceed to send ${amt} to {rcvr}. We need to find the recipient's email. Let's check the list of APIs to see how we can get contact details of a user.
answer = world.apis.api_docs.show_api_descriptions(app_name='venmo')""".format(amt = amount,rcvr = recipient)}]},
        
        {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
{apis}""".format(q = user_msg, apis = api_list)}, 
                      {"role": "assistant", "content": """#The 'search_friends' API seems to be the one we need to get the recipient's email. Let's check its specifications.
answer = world.apis.api_docs.show_api_doc(app_name='venmo', api_name='search_friends')"""}]},
        
        {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
{frnd_detail}""".format(q = user_msg, frnd_detail = search_friends_details)}, 
                      {"role": "assistant", "content": """#Let's find {rcvr}'s email using the 'serach_friends' API.
answer = world.apis.spotify.search_friends(access_token = access_token, query = "{rcvr}")""".format(rcvr = recipient)}]}
    ]
    
    try:
        note = example['slots']['note']
        
        temp = {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
""".format(q = user_msg) + 
str({'first_name': recipient, 'last_name': '', 'email': email, 'registered_at': '', 'friends_since': ''})}, 
                      {"role": "assistant", "content": """#{rcvr}'s email is {em}. Let's proceed to send ${amt} for {nt} using the 'create_transaction' API.
answer = world.apis.venmo.create_transaction( receiver_email='{em}', amount={amt}, description={nt}))""".format(rcvr = recipient, em = email, amt = amount, nt = note)}]}
        
        message_list.append(temp)
        
    except:

        temp = {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
""".format(q = user_msg) + 
str({'first_name': recipient, 'last_name': '', 'email': email, 'registered_at': '', 'friends_since': ''})}, 
                      {"role": "assistant", "content": """#{rcvr}'s email is {em}. Let's proceed to send ${amt} using the 'create_transaction' API.
answer = world.apis.venmo.create_transaction( receiver_email='{em}', amount={amt}))""".format(rcvr = recipient, em = email, amt = amount)}]}

        message_list.append(temp)
        
     
    message_list.append(
        {"messages": [{"role": "user", "content": """User Instruction: {q}
Current Observation:
{"message":"Sent money", "transaction_id": ""}"""}, 
                      {"role": "assistant", "content": """#The transaction was successful. Now, let's mark the task as complete.
answer = world.apis.supervisor.complete_task()"""}]}
    )
    
    
    return message_list
            

In [20]:
def build_dataset(user_queries, output_path):
    with open(output_path, 'w') as f:
        for query in user_queries:
            traj_list = generate_trajectory(query)
            for it in traj_list:
                json.dump(it, f)
                f.write('\n')

In [21]:
build_dataset(user_queries, '/projects/asankaranarayana/Fine_Tune_Hackathon/v1/task_trajectories_v2.jsonl')